In [2]:
# 模型构造
# http://zh.gluon.ai/chapter_gluon-basics/block.html

# 多层感知机的计算
from mxnet import nd
from mxnet.gluon import nn

net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(256, activation='relu'))
    net.add(nn.Dense(10))
print(net)

/home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Sequential(
  (0): Dense(None -> 256, Activation(relu))
  (1): Dense(None -> 10, linear)
)


In [3]:
net.initialize()
x = nd.random.uniform(shape=(4, 20))
y = net(x)
print(net)
print(y)

Sequential(
  (0): Dense(20 -> 256, Activation(relu))
  (1): Dense(256 -> 10, linear)
)

[[ 0.03126615  0.04562764  0.00039857 -0.08772386 -0.05355632  0.02904574
   0.08102557 -0.01433946 -0.04224151  0.06047882]
 [ 0.02871901  0.03652265  0.00630051 -0.05650971 -0.07189322  0.08615957
   0.05951559 -0.06045965 -0.0299026   0.05651001]
 [ 0.02147349  0.04818896  0.05321142 -0.12616856 -0.0685023   0.09096345
   0.04064304 -0.05064794 -0.02200242  0.04859561]
 [ 0.03780478  0.0751239   0.03290457 -0.11641113 -0.03254967  0.0586529
   0.02542157 -0.01697343 -0.00049652  0.05892839]]
<NDArray 4x10 @cpu(0)>


In [11]:
# 使用 nn.Block 构造模型
class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense0 = nn.Dense(256)
            self.dense1 = nn.Dense(10)
    def forward(self, x):
        return self.dense1(nd.relu(self.dense0(x)))

In [13]:
net2 = MLP()
print(net2)
net2.initialize()
x = nd.random.uniform(shape=(4, 20))
y = net2(x)
y

MLP(
  (dense0): Dense(None -> 256, linear)
  (dense1): Dense(None -> 10, linear)
)



[[ 0.02611046 -0.00898641  0.04863964 -0.03597313  0.08567166 -0.03285579
   0.00181929 -0.02620459  0.01748283 -0.00853055]
 [ 0.00497482  0.01554187  0.04043619 -0.02491888  0.0717035  -0.0377518
   0.00745001  0.0081154   0.03080349 -0.00300385]
 [ 0.01212909 -0.02291088  0.05410973 -0.0643751   0.08853698 -0.02997083
  -0.02801304 -0.00600521  0.03243509 -0.00871116]
 [ 0.01189529 -0.00181999  0.02517185  0.00576071  0.03568135 -0.04611992
   0.03752039 -0.00957203  0.01495805 -0.0060389 ]]
<NDArray 4x10 @cpu(0)>

In [14]:
nn.Dense

mxnet.gluon.nn.basic_layers.Dense

In [16]:
net3 = MLP(prefix='another_mlp_')
print('customized prefix:', net3.dense0.name)
print(net3)

customized prefix: another_mlp_dense0
MLP(
  (dense0): Dense(None -> 256, linear)
  (dense1): Dense(None -> 10, linear)
)


In [20]:
# nn.Sequential
class Sequential2(nn.Block):
    def __init__(self, **kwargs):
        super(Sequential2, self).__init__(**kwargs)
    def add(self, block):
        self._children.append(block)
    def forward(self, x):
        for block in self._children:
            x = block(x)
        return x

In [22]:
net4 = Sequential2()
with net.name_scope():
    net4.add(nn.Dense(256, activation='relu'))
    net4.add(nn.Dense(10))
net4.initialize()
y = net4(x)
y


[[-0.02542969  0.01105447 -0.02126193  0.01974962 -0.02856903  0.04831402
  -0.00491117  0.03298518 -0.08613102 -0.05270944]
 [-0.02791293  0.02870862  0.01503616  0.00348095 -0.00159181 -0.0207642
   0.02418825  0.04087722 -0.07049553 -0.06113587]
 [-0.0341648  -0.02236486  0.03288743  0.00123798 -0.01106082 -0.00410438
   0.04807579  0.04631495 -0.08527313 -0.04669718]
 [-0.03053684  0.04171332  0.00441197 -0.00048724 -0.02169376  0.0202373
  -0.00683418  0.06006115 -0.05523611 -0.01838877]]
<NDArray 4x10 @cpu(0)>

In [23]:
class FancyMLP(nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense = nn.Dense(256)
            self.weight = nd.random_uniform(shape=(256, 20))
    def forward(self, x):
        x = nd.relu(self.dense(x))
        x = nd.relu(nd.dot(x, self.weight) + 1)
        x = nd.relu(self.dense(x))
        return x

In [24]:
fancy_mlp = FancyMLP()
fancy_mlp.initialize()
y = fancy_mlp(x)
print(y.shape)

(4, 256)


In [25]:
# nn.Block和nn.Sequential的嵌套使用
class RecMLP(nn.Block):
    def __init__(self, **kwargs):
        super(RecMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        with self.name_scope():
            self.net.add(nn.Dense(256, activation='relu'))
            self.net.add(nn.Dense(128, activation='relu'))
            self.dense = nn.Dense(64)
    def forward(self, x):
        return nd.relu(self.dense(self.net(x)))

rec_mlp = nn.Sequential()
rec_mlp.add(RecMLP())
rec_mlp.add(nn.Dense(10))
print(rec_mlp)

Sequential(
  (0): RecMLP(
    (net): Sequential(
      (0): Dense(None -> 256, Activation(relu))
      (1): Dense(None -> 128, Activation(relu))
    )
    (dense): Dense(None -> 64, linear)
  )
  (1): Dense(None -> 10, linear)
)
